In [50]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, accuracy_score
# plt.style.use('seaborn-whitegrid')

In [51]:
entidades = pd.read_csv('../data/silver/entidades_unicas.csv', sep=';')
entidades = entidades.drop(columns=['DEPARTAMENTO', 'MUNICIPIO', 'ORDEN'])
print(entidades.shape)
entidades.head(2)

(10976, 4)


,NOMBRE,CODIGO,NIT,ID
0,( SECRETARIA DISTRITAL INTEGRACION SOCIAL ),702271321.0,8999990619,0
1,INSTITUTO COLOMBIANO AGROPECUARIO SECCIONAL CU...,709368088.0,899999069,0


Se reducirá el universo de estudio para tener en cuenta las 10 categorías con más datos.

In [52]:
entidades['ID'].value_counts()[:10]

ID
0       120
2323     94
30       37
3747     37
1177     33
93       30
2584     27
53       24
104      24
142      21
Name: count, dtype: int64

In [53]:
entidades_model = entidades[entidades['ID'].isin(list(entidades['ID'].value_counts()[:100].reset_index()['ID']))]
print(entidades_model.shape)
entidades_model.head(2)

(1432, 4)


,NOMBRE,CODIGO,NIT,ID
0,( SECRETARIA DISTRITAL INTEGRACION SOCIAL ),702271321.0,8999990619,0
1,INSTITUTO COLOMBIANO AGROPECUARIO SECCIONAL CU...,709368088.0,899999069,0


In [54]:
entidades_model['feature'] = entidades_model.apply(lambda row: ' | '.join(row.values.astype(str)), axis=1)
entidades_model = entidades_model[['feature', 'ID']]

/tmp/ipykernel_14489/1243066781.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entidades_model['feature'] = entidades_model.apply(lambda row: ' | '.join(row.values.astype(str)), axis=1)


In [55]:
entidades_model.head(2)

,feature,ID
0,( SECRETARIA DISTRITAL INTEGRACION SOCIAL ) | ...,0
1,INSTITUTO COLOMBIANO AGROPECUARIO SECCIONAL CU...,0


In [56]:
# Uso 70% para entrenamiento (random split)
train_df = entidades_model.sample(frac=0.7,random_state=200)
rest_df = entidades_model.drop(train_df.index)
# Uso 15% para validacion y 15% para test
val_df=rest_df.sample(frac=0.5,random_state=200)
test_df=rest_df.drop(val_df.index)

In [57]:
print('train_df:', train_df.shape)
print('val_df:', val_df.shape)
print('test_df:', test_df.shape)

train_df: (1002, 2)
val_df: (215, 2)
test_df: (215, 2)


In [58]:
X_train, Y_train = train_df['feature'], train_df['ID']
X_val, Y_val = val_df['feature'], val_df['ID']
X_test, Y_test = test_df['feature'], test_df['ID']

In [59]:
model = make_pipeline(TfidfVectorizer(), MLPClassifier())
model.fit(X_train, Y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('mlpclassifier', MLPClassifier())])

In [62]:
labels_test = model.predict(X_test)
accuracy_score(Y_test, labels_test)

0.9767441860465116

In [63]:
labels_val = model.predict(X_val)
accuracy_score(Y_val, labels_val)

0.958139534883721

In [67]:
def predict_category(s, train=X_train, model=model):
    pred = model.predict([s])
    return pred[0]

In [98]:
predict_category('SNEA')

0

In [100]:
model.get_params()

{'memory': None,
 'steps': [('tfidfvectorizer', TfidfVectorizer()),
  ('mlpclassifier', MLPClassifier())],
 'verbose': False,
 'tfidfvectorizer': TfidfVectorizer(),
 'mlpclassifier': MLPClassifier(),
 'tfidfvectorizer__analyzer': 'word',
 'tfidfvectorizer__binary': False,
 'tfidfvectorizer__decode_error': 'strict',
 'tfidfvectorizer__dtype': numpy.float64,
 'tfidfvectorizer__encoding': 'utf-8',
 'tfidfvectorizer__input': 'content',
 'tfidfvectorizer__lowercase': True,
 'tfidfvectorizer__max_df': 1.0,
 'tfidfvectorizer__max_features': None,
 'tfidfvectorizer__min_df': 1,
 'tfidfvectorizer__ngram_range': (1, 1),
 'tfidfvectorizer__norm': 'l2',
 'tfidfvectorizer__preprocessor': None,
 'tfidfvectorizer__smooth_idf': True,
 'tfidfvectorizer__stop_words': None,
 'tfidfvectorizer__strip_accents': None,
 'tfidfvectorizer__sublinear_tf': False,
 'tfidfvectorizer__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidfvectorizer__tokenizer': None,
 'tfidfvectorizer__use_idf': True,
 'tfidfvectorizer__vocab